In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from keras.backend import clear_session
to_categorical = tf.keras.utils.to_categorical

array_to_img = tf.keras.preprocessing.image.array_to_img
ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px

import numpy as np
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score

ModuleNotFoundError: No module named 'tensorflow.python'

In [3]:
clear_session()

In [4]:
img_rows, img_cols = 1024, 1024
batch_size = 16
epochs = 100
num_classes = 2
seed = 42

input_shape = (img_rows, img_cols, 3)

In [5]:
datagen = ImageDataGenerator(rescale = 1./255,
                             horizontal_flip = True,
                             zoom_range=0.3,
                             shear_range = 0.2,
                             rotation_range=0.3)

val_test_datagen = ImageDataGenerator(rescale = 1./255)

### Data

In [6]:
train_it = datagen.flow_from_directory('/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 1/dataset/train',
                                       target_size=(img_rows, img_cols),
                                       batch_size=batch_size,
                                       class_mode='categorical',
                                       shuffle=True,
                                       seed = seed)

Found 254 images belonging to 2 classes.


In [7]:
val_it = val_test_datagen.flow_from_directory('/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 1/dataset/valid',
                                              target_size=(img_rows, img_cols),
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=False,
                                              seed = seed)

test_it = val_test_datagen.flow_from_directory('/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 1/dataset/test',
                                               target_size=(img_rows, img_cols),
                                               batch_size=batch_size,
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed = seed)

Found 80 images belonging to 2 classes.
Found 114 images belonging to 2 classes.


In [8]:
labels = list(train_it.class_indices.keys())
labels

['abnormal', 'normal']

In [9]:
Counter(train_it.classes)

Counter({0: 168, 1: 86})

In [10]:
print(f'Total augmented abnormal image: {Counter(train_it.classes)[0]*100}')
print(f'Total augmented normal image: {Counter(train_it.classes)[1]*100}')

Total augmented abnormal image: 16800
Total augmented normal image: 8600


In [11]:
Counter(test_it.classes)

Counter({0: 57, 1: 57})

In [12]:
Counter(val_it.classes)

Counter({0: 56, 1: 24})

In [13]:
weights = compute_class_weight('balanced',
                               classes = np.unique(train_it.classes),
                               y = train_it.classes)
weights

array([0.75595238, 1.47674419])

In [14]:
cw = dict(zip(np.unique(train_it.classes), weights))
cw

{0: 0.7559523809523809, 1: 1.4767441860465116}

In [15]:
#Feature Extraction
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

#Image Classification
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Train

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 1022, 1022, 32)      │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 511, 511, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 509, 509, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 254, 254, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 252, 252, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 126, 126, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 124, 124, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 246016)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      31,490,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,564,130 (120.41 MB)

 Trainable params: 31,564,130 (120.41 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
STEP_SIZE_TRAIN=train_it.n//train_it.batch_size
STEP_SIZE_VALID=val_it.n//val_it.batch_size
STEP_SIZE_TEST=test_it.n//test_it.batch_size

In [18]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience = 2, verbose=1,factor=0.1, min_lr=0.000001)
callbacks_list = [learning_rate_reduction]

In [ ]:
his = model.fit(
        train_it,
        validation_data=val_it,
        epochs=epochs,
        class_weight=cw,
        callbacks=callbacks_list,
        shuffle=True,
        steps_per_epoch=STEP_SIZE_TRAIN,
        validation_steps=STEP_SIZE_VALID)

In [19]:
his = model.fit(
    train_it,
    validation_data=val_it,
    epochs=epochs,
    class_weight=cw,
    callbacks=callbacks_list,
    steps_per_epoch=None,
    validation_steps=None
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.6099 - loss: 1.0551 - val_accuracy: 0.8750 - val_loss: 0.2823 - learning_rate: 0.0010
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.8922 - loss: 0.4295 - val_accuracy: 0.8125 - val_loss: 0.4377 - learning_rate: 0.0010
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.8596 - loss: 0.2839 - val_accuracy: 0.9500 - val_loss: 0.1808 - learning_rate: 0.0010
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9108 - loss: 0.1884 - val_accuracy: 0.9375 - val_loss: 0.1693 - learning_rate: 0.0010
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9188 - loss: 0.1656 - val_accuracy: 0.9625 - val_loss: 0.0783 - learning_rate: 0.0010
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9889 - loss: 0.0553 - val_accuracy: 0.9750 - val_loss: 0.0792 - learning_rate: 0.0010
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9543 - loss: 0.1961
Epoch 7: ReduceLROnPla


### Evaluate

In [20]:
lossacc = model.evaluate(test_it)
print(f'Model 1 Loss : {lossacc[0]}')
print(f'Model 1 Accuracy : {lossacc[1]}')

8/8 ━━━━━━━━━━━━━━━━━━━━ 80s 11s/step - accuracy: 0.9251 - loss: 0.2302
Model 1 Loss : 0.1692088395357132
Model 1 Accuracy : 0.9473684430122375


In [21]:
h1 = go.Scatter(y=his.history['loss'],
                    mode="lines",
                    line=dict(
                        width=2,
                        color='blue'),
                        name="loss"
                   )
h2 = go.Scatter(y=his.history['val_loss'],
                    mode="lines",
                    line=dict(
                        width=2,
                        color='red'),
                        name="val_loss"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data, layout=layout1)
plotly.offline.iplot(fig1)

In [22]:
h1 = go.Scatter(y=his.history['accuracy'],
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="acc"
                   )
h2 = go.Scatter(y=his.history['val_accuracy'],
                    mode="lines", line=dict(
                    width=2,
                    color='red'),
                    name="val_acc"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Accuracy',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1)

In [23]:
predicted_score = model.predict(test_it, verbose=1)
predicted_classes = np.argmax(predicted_score, axis=1)

8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 377ms/step


In [24]:
def cm_plot(cm, labels):
    x = labels
    y = labels

    z_text = [[str(y) for y in x] for x in cm]
    fig = ff.create_annotated_heatmap(cm, x=x, y=y, annotation_text=z_text, colorscale='blues')

    fig.update_layout(title_text='Confusion Matrix')

    fig.add_annotation(dict(font=dict(color="black",size=13),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted Value",
                            xref="paper",
                            yref="paper"
                           ))

    fig.add_annotation(dict(font=dict(color="black",size=13),
                            x=-0.20,
                            y=0.5,
                            showarrow=False,
                            text="Real Value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"
                           ))

    fig.update_layout(margin=dict(t=50, l=200))
    fig['layout']['yaxis']['autorange'] = "reversed"

    fig['data'][0]['showscale'] = True
    fig.show()

In [25]:
cm = confusion_matrix(test_it.classes, predicted_classes)
print(cm)
cm_plot(cm, labels)

[[52  5]
 [ 1 56]]


In [26]:
report = classification_report(test_it.classes, predicted_classes, target_names=labels, digits=4)

print(report)

              precision    recall  f1-score   support

    abnormal     0.9811    0.9123    0.9455        57
      normal     0.9180    0.9825    0.9492        57

    accuracy                         0.9474       114
   macro avg     0.9496    0.9474    0.9473       114
weighted avg     0.9496    0.9474    0.9473       114



### Save

In [27]:
model.save('/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 1/model_1_16b100e_9_11_2024.keras')
model.save('/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 1/model_1_16b100e_9_11_2024.h5')

In [28]:
import pickle as p
his_model_name = '/content/drive/MyDrive/640710819_Projects/Tongue Classification/model 1/model_1_16b100e_9_11_2024_history'
with open(his_model_name, 'wb') as file:
  p.dump(his.history, file)